In [1]:
import numpy as np
import cv2

In [ ]:
## haar cascade (고양이 얼굴 검출기)

cat_face_cascade = cv2.CascadeClassifier('haarcascade_frontalcatface_extended.xml')

img = cv2.imread('image/cats.jpg', 1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cat_faces = cat_face_cascade.detectMultiScale(gray, 1.1, 3)


for (x,y,w,h) in cat_faces :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## haar cascade (눈, 얼굴 검출기)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

img = cv2.imread('image/aespa.jpg', 1)
img = cv2.resize(img, dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.1, 3)
eyes = eye_cascade.detectMultiScale(gray, 1.1, 3)


for (x,y,w,h) in faces :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    
    
for (x,y,w,h) in eyes :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
## ORB 알고리즘(Oriented fast and Rotated Breif)

src = cv2.imread('image/apple_book.jpg', 1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

target = cv2.imread('image/apple.jpg', 0)

orb = cv2.ORB_create(
    nfeatures = 40000,
    scaleFactor = 1.2,
    nlevels = 8,
    edgeThreshold = 31,
    firstLevel = 0,
    WTA_K = 2,
    scoreType = cv2.ORB_HARRIS_SCORE,
    patchSize = 31,
    fastThreshold = 20,
    
)

kp1, des1 = orb.detectAndCompute(gray, None)
kp2, des2 = orb.detectAndCompute(target, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x: x.distance)

for i in matches[:100]:
    idx = i.queryIdx
    x1, y1, =  kp1[idx].pt
    cv2.circle(src, (int(x1), int(y1)), 3, (255, 0, 0), 3)
    
cv2.imshow('src', src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
## K - means clustering (평균 군집화)

src = cv2.imread('image/flower.jpg', 1)

data = src.reshape(-1,3).astype(np.float32)
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)
retval, bestLabels, centers = cv2.kmeans(data, 5, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = centers.astype(np.uint8)
dst = centers[bestLabels].reshape(src.shape)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()